# Metrics - NPS (for ABSA, not topic modelling)

In [1]:
import duckdb as ddb
import sys
from pathlib import Path
common_code_path = Path.cwd().parent.parent / "plp_common_code"
print(common_code_path)

if str(common_code_path) not in sys.path:
    sys.path.append(str(common_code_path))
from amhere import foundme
foundme()  # this fn will fail if common code path is not found or !exists
del foundme
del common_code_path

cwd = Path.cwd()
db_path = cwd / 'database'
db_file_name = 'aspect_topic_db.parquet'  # db path
database_path_file = db_path / db_file_name
# self._output_path = cwd / 'data_output'
print(database_path_file)

D:\OneDrive\RChai\Documents\NUS_ISS_Prjs\4_PLP\06_Code_Actuals\plp_common_code
D:\OneDrive\RChai\Documents\NUS_ISS_Prjs\4_PLP\06_Code_Actuals\3.4_Components_Deployed_Soln\3.4.8_Cust_Metrics\database\aspect_topic_db.parquet


In [2]:
df = ddb.read_parquet(str(database_path_file)).to_df()
df.columns

Index(['date', 'dlg_id', 'text', 'product', 'action', 'sentiment',
       'aspect_topic'],
      dtype='object')

In [3]:
# dft = ddb.query("""select count(*) as num_rows from df;""").to_df()
# dft

# Get subset data by date range

In [4]:
start_date = '2024-10-05'
end_date = '2024-10-05'

sql_query = f"""
select * from df where date >= '{start_date}' and date <= '{end_date}';
"""
dft = ddb.query(sql_query).to_df()
display(dft.head(3))
display(dft.tail(3))

date              dlg_id  \
0  2024-10-05  2024-10-07_1017--0   
1  2024-10-05  2024-10-07_1017--1   
2  2024-10-05  2024-10-07_1017--2   

                                                text product   action  \
0                      i am still waiting on my card    card  inquire   
1  what can i do if my card still hasnt arrived a...    card  inquire   
2  i have been waiting over a week is the card st...    card  inquire   

  sentiment   aspect_topic  
0   neutral  inquire__card  
1   neutral  inquire__card  
2   neutral  inquire__card

date               dlg_id  \
13971  2024-10-05  2024-10-14__dlg_992   
13972  2024-10-05  2024-10-14__dlg_992   
13973  2024-10-05  2024-10-14__dlg_992   

                                                    text     product   action  \
13971  Hi Alex, I hope you're doing well. I'm very im...        loan    block   
13972  Sure, the website is [Website Name]. While the...        loan  dispute   
13973  That would be perfect. Please go ahead and set...  debit_card   access   

           sentiment        aspect_topic  
13971  very positive         block__loan  
13972  very negative       dispute__loan  
13973  very negative  access__debit_card

# Convert Sentiment to Score

In [5]:
def convert_sentiment_to_score(x):
    x = x.lower()
    match x:
        case 'very negative':
            return 0
        case 'negative':
            return 2.5
        case 'neutral':
            return 5
        case 'positive':
            return 7.5
        case 'very positive':
            return 10
        case _:
            raise SystemError('sentiment polarity is unknown value')
    
dft['sentiment_score'] = dft['sentiment'].apply(lambda x: convert_sentiment_to_score(x))
dft[['sentiment', 'sentiment_score']].sample(5)

sentiment  sentiment_score
13741  very positive             10.0
876         positive              7.5
8366   very positive             10.0
7783   very positive             10.0
9415        positive              7.5

In [6]:
sql_query = f"""
select 
aspect_topic 
,avg(sentiment_score) as avg_sentiment_score 
,case
    when avg_sentiment_score >= 9 then 'promoters'
    when avg_sentiment_score >= 7 then 'passives'
    else 'detractors'
end as nps_group 
, ANY_VALUE(dlg_id) as rep_dlg_id -- representative dialog id
, ANY_VALUE(text) as rep_text  -- representative text
from dft
group by aspect_topic
order by avg_sentiment_score desc, aspect_topic;
"""
df_nps_groups = ddb.query(sql_query).to_df()
df_nps_groups.sample(5)

aspect_topic  avg_sentiment_score   nps_group  \
92           update__atm             6.309524  detractors   
348       apply__website             4.021739  detractors   
211  open__forex_account             5.312500  detractors   
235      access__account             5.113636  detractors   
374     unblock__website             3.500000  detractors   

                rep_dlg_id                                           rep_text  
92   2024-10-07_1017--3530  if i want to change my pin number can i direct...  
348   2024-10-14__dlg_1230  I think applying online would be convenient fo...  
211   2024-10-14__dlg_1100  I want to open a forex account that has compet...  
235  2024-10-07_1017--2859  i have an emergency and lost my phone  i need ...  
374   2024-10-14__dlg_1906  This is ridiculous. The last transaction shoul...

### Calculate NPS for Bank

In [7]:
sql_query = """
with 
respondents as (select count(*) as total_respondents from df_nps_groups)
,promoters as (select count(*) as num_promoters from df_nps_groups where nps_group = 'promoters')
, detractors as (select count(*) as num_detractors from df_nps_groups where nps_group = 'detractors')
select
r.total_respondents
,p.num_promoters
,(p.num_promoters / r.total_respondents * 100) as pct_promoters
,d.num_detractors
,(d.num_detractors / r.total_respondents * 100) as pct_detractors
,(pct_promoters - pct_detractors) as nps -- can range from -100 to +100
from respondents as r, promoters as p, detractors as d
;
"""
df_nps = ddb.query(sql_query).to_df()
df_nps  # the nps score

total_respondents  num_promoters  pct_promoters  num_detractors  \
0                393              5       1.272265             352   

   pct_detractors        nps  
0        89.56743 -88.295165